In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import cPickle as pk
import pandas as pd

# Code combining into Auth Year combination sets

##### Prelim Data Munge

In [7]:
#Date to years function - works on application date and publication date in one
def date_to_year(df):
    if 'appln_filing_date' in df.columns:
        df.appln_filing_date = df.appln_filing_date.map(lambda x:int(x[:4]))
        df.rename(columns={'appln_filing_date':'app_year'}, inplace=True)
    if 'publn_date'in df.columns:
        df.publn_date = df.publn_date.map(lambda x:int(x[:4]))
        df.rename(columns={'publn_date':'pub_year'}, inplace=True)
    return

## Importing
[to top](#Table-of-Contents) 

In [8]:
pwd

u'/Users/innovation-sfi/Dropbox/Research/Innovation/EPO/Data_processing/Python'

In [9]:
#%time df1 = pk.load(open('pickles_and_munged_csv/final_data_v3.pk','rb'))
%time df2 = pd.read_csv('pickles_and_munged_csv/final_data_codes_as_ints_v1.csv', engine='c', dtype={'publn_nr':str,'appln_id':str})
# pickling is so so slow...

CPU times: user 15.3 s, sys: 1.96 s, total: 17.3 s
Wall time: 17.3 s


In [10]:
# take just top 10 countries by patent count for starters
fin_top10 = pk.load(open('pickles_and_munged_csv/final_top10_v1.pk','rb'))
fin_top10

array(['JP', 'CN', 'US', 'DE', 'KR', 'FR', 'SU', 'GB', 'RU', 'TW', 'CH'], dtype=object)

In [11]:
# munge dates to years
date_to_year(df2)

In [12]:
df2 = df2[df2.appln_auth.isin(fin_top10)]

# Group - Apply - Combine: Auth:Year:ID, 

In [13]:
# Group
grp_ayi = df2.groupby(['appln_auth', 'pub_year', 'appln_id'])

In [14]:
# Apply - combine codes for each app into list NOTE time is for 100000 
%time app_id_class_ls =  grp_ayi['ipc_class_symbol'].apply(list); app_id_class_ls.head()

CPU times: user 8min 25s, sys: 10.8 s, total: 8min 36s
Wall time: 8min 36s


In [15]:
# regroup by just auth - year
grp_ay = app_id_class_ls.groupby(level=[0,1])

In [20]:
# Apply 2 - combine lists of codes for each year into list
au_yr_ls_ls = grp_ay.apply(list); au_yr_ls_ls.head()

appln_auth  pub_year
CH          1900                                  [[6088, 6118]]
            1903                                  [[6069, 7114]]
            1904                      [[4364, 4364, 4367, 4367]]
            1905        [[6948, 7165], [3331, 3333, 3333, 3333]]
            1906                                  [[6114, 6115]]
dtype: object

In [35]:
yr_lsls_au = au_yr_ls_ls.reset_index(level=0).pivot(columns='appln_auth'); yr_lsls_au
yr_lsls_au.columns = yr_lsls_au.columns.droplevel(); yr_lsls_au.head()

appln_auth,CH,CN,DE,FR,GB,JP,KR,RU,SU,TW,US
pub_year,,,,,,,,,,,
1836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[1512, 1788], [1267, 1854], [1761, 1820], [24..."
1837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[5829, 5831], [4903, 5435], [799, 5643], [602..."
1838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[1536, 1773], [1153, 1207], [2722, 2746], [40..."
1839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[4208, 4227], [5074, 6174], [4911, 4997], [33..."
1840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[2720, 2746], [1861, 1861], [1160, 5980], [48..."


In [36]:
def map_nan_lsls(ls):
    if type(ls) != type([]):
        ls = []
    return ls

In [37]:
yr_lsls_au = yr_lsls_au.applymap(map_nan_lsls); yr_lsls_au.head()

appln_auth,CH,CN,DE,FR,GB,JP,KR,RU,SU,TW,US
pub_year,,,,,,,,,,,
1836,[],[],[],[],[],[],[],[],[],[],"[[1512, 1788], [1267, 1854], [1761, 1820], [24..."
1837,[],[],[],[],[],[],[],[],[],[],"[[5829, 5831], [4903, 5435], [799, 5643], [602..."
1838,[],[],[],[],[],[],[],[],[],[],"[[1536, 1773], [1153, 1207], [2722, 2746], [40..."
1839,[],[],[],[],[],[],[],[],[],[],"[[4208, 4227], [5074, 6174], [4911, 4997], [33..."
1840,[],[],[],[],[],[],[],[],[],[],"[[2720, 2746], [1861, 1861], [1160, 5980], [48..."


In [38]:
yr_lsls_au.applymap(len)

appln_auth,CH,CN,DE,FR,GB,JP,KR,RU,SU,TW,US
pub_year,,,,,,,,,,,
1836,0,0,0,0,0,0,0,0,0,0,7
1837,0,0,0,0,0,0,0,0,0,0,28
1838,0,0,0,0,0,0,0,0,0,0,38
1839,0,0,0,0,0,0,0,0,0,0,25
1840,0,0,0,0,0,0,0,0,0,0,35
1841,0,0,0,0,0,0,0,0,0,0,40
1842,0,0,0,0,0,0,0,0,0,0,47
1843,0,0,0,0,0,0,0,0,0,0,48
1844,0,0,0,0,0,0,0,0,0,0,46


# Above is THE data frame for building networks
NetworkX can take the list of lists and add links for all permuations of the items in each list
The idea being that we go from dataframe with each value being an encoding of the adjaceny matric for each country for that year, then apply a mapping to each value that's a function using networkx to turn that into a graph, thus leaving the same data frame but with nx graphs instead of lists of lists. Then this will become the root dataframe. 
## Calling this type of dataframe a code sets frame

In [40]:
for col in yr_lsls_au:
    print col
    for inx in yr_lsls_au.index[1:]:
        print inx,
        yr_lsls_au.loc[inx,col] = yr_lsls_au.loc[inx,col] + yr_lsls_au.loc[inx-1,col]

CH
1837 1838 1839 1840 1841 1842 1843 1844 1845 1846 1847 1848 1849 1850 1851 1852 1853 1854 1855 1856 1857 1858 1859 1860 1861 1862 1863 1864 1865 1866 1867 1868 1869 1870 1871 1872 1873 1874 1875 1876 1877 1878 1879 1880 1881 1882 1883 1884 1885 1886 1887 1888 1889 1890 1891 1892 1893 1894 1895 1896 1897 1898 1899 1900 1901 1902 1903 1904 1905 1906 1907 1908 1909 1910 1911 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 CN
1837 1838 1839 1840 1841 1842 1843 1844 1845 1846 1847 1848 1849 1850 1851 1852 1853 1854 1855 1856

In [43]:
# gotta pickle it, writing to csv doesn't preserve the columns and index properly...
pk.dump(yr_lsls_au, open('pickles_and_munged_csv/top11_cated_cumed_codes_as_ints_v1.pk', 'wb'))

In [42]:
yr_lsls_au.applymap(len)

appln_auth,CH,CN,DE,FR,GB,JP,KR,RU,SU,TW,US
pub_year,,,,,,,,,,,
1836,0,0,0,0,0,0,0,0,0,0,7
1837,0,0,0,0,0,0,0,0,0,0,35
1838,0,0,0,0,0,0,0,0,0,0,73
1839,0,0,0,0,0,0,0,0,0,0,98
1840,0,0,0,0,0,0,0,0,0,0,133
1841,0,0,0,0,0,0,0,0,0,0,173
1842,0,0,0,0,0,0,0,0,0,0,220
1843,0,0,0,0,0,0,0,0,0,0,268
1844,0,0,0,0,0,0,0,0,0,0,314
